
# Neural Architecture - EVA5


## *7 Steps to build a DNN using Pytorch*

--------

This code provides a simple steps to coding a convolutional neural network using pytorch.

Coding with pytorch is becomes quite easy if we come to terms the building blocks of DNN and how they are put to gether. 

The list of items, if followed would cover 90% of any DNN using pytorch. The details of each program is provided as a comment or additional text. 




1.   Import relavant libraries for pytorch (nn, functional, optim, datasets, transforms). 
2.   Class functions for the neural architecture. 
3.   View model summary
4.   Load the data (torch dataset) and use "dataloader" to transform (tensor, normalize, batches) the data
5.   Train the model (function): for each batch 
     
     a. Load the data to device

     b. forward propagate

     c. Calculate Loss (function)

     d. Back propagate

     e. Update parameters

     f. Calculate Accuracy per epoch

6.   Test/evaluate the model (function): Test set batch
     
     a. Load the data to device

     b. forward propagate

     c. Predict and calculate accuracy


7.  Decide on epochs and loop through them:

    a. Call the train function

    b. call the test function





In [ ]:
from __future__ import print_function
import torch #import torch
import torch.nn as nn # neural network library from torch
import torch.nn.functional as F # provides many functions that work like the modules we find in nn
import torch.optim as optim # calls optimizer that helps adjusting the model parameters(weights) example SGD, Adam etc.,
from torchvision import datasets, transforms # torch has inbuild datasets that can be explored. 

"__future__"is a pseudo-module which programmers can use to enable new language features which are not compatible with the current interpreter.

https://stackoverflow.com/questions/7075082/what-is-future-in-python-used-for-and-how-when-to-use-it-and-how-it-works/7075121


In [ ]:
# We build a neural architecture by sublassing nn.Module. 
#we use a subclass of nn.Module to contain our entire model. We could also use
#subclasses to define new building blocks for more complex networks. 

# Typically, our computation will use other modules—premade like convolutions
# or customized. To include these submodules, we typically define them in the constructor
# __init__ and assign them to self for use in the forward function.
# They will, at the same time, hold their parameters throughout the lifetime of our module.
# Note that you need to call super().__init__() before you can do that (or PyTorch will remind you)

class Net(nn.Module): # Initialize a class Net inheriting torch's nn.Module
    def __init__(self):
        super(Net, self).__init__() # Super init is initallized so that nn.Module method is is run. Else a error is thrown
                                                      # Input    | conv          | output   |Receptive field
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   # 28*28*1  | (3*3*1)*32    | 28*28*32 |RF 3*3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 28*28*32 | (3*3*32)*64   | 28*28*64 |RF 5*5
        self.pool1 = nn.MaxPool2d(2, 2)               # 28*28*64 | Max pooling   | 14*14*64 |RF 10*10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # 14*14*64 | (3*3*64)*128  | 14*14*128|RF 12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)# 14*14*128| (3*3*128)*256 | 14*14*256|RF 14*14
        self.pool2 = nn.MaxPool2d(2, 2)               # 14*14*256| Max Pooling   | 7*7*256  |RF 28*28
        self.conv5 = nn.Conv2d(256, 512, 3)           # 7*7*256  | (3*3*256)*512 | 5*5*512  |RF 30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)          # 5*5*512  | (3*3*512)*1024| 3*3*1024 |RF 32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)           # 3*3*1024 | (3*3*1024)*10 | 10*10*1  |RF 34*34

# seperate function to keep the Functional transactions of nn module for forward propagation.
    """forward
    Defines the computation performed at every call.
    Args:
        x: the input
    Returns:
        log_softmax(x)
    """ 
    def forward(self, x): # forward function
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #pool1 -> relu -> conv2 -> relu -> conv1 (x)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #pool2 -> relu -> conv4 -> relu -> conv3 
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             # relu -> conv6 -> relu -> conv5 
        x = F.relu(self.conv7(x))                                 # relu -> conv7
        x = x.view(-1, 10)                                        # reshape and view as a vector of 10
        return F.log_softmax(x)                                   # return softmax applied to the vector. 

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)